# Final Project Template 

## 1) Get your data
You may use any data set(s) you like, so long as they meet these criteria:

* Your data must be publically available for free.
* Your data should be interesting to _you_. You want your final project to be something you're proud of.
* Your data should be "big enough":
    - It should have at least 1,000 rows.
    - It should have enough of columns to be interesting.
    - If you have questions, contact a member of the instructional team.

## 2) Provide a link to your data
Your data is required to be free and open to anyone.
As such, you should have a URL which anyone can use to download your data:

In [1]:
# https://www.ncbi.nlm.nih.gov/pubmed/

# 3) Import your data

In the space below, import your data. If your data span multiple files, read them all in. If applicable, merge or append them as needed.

In [2]:
#importing needed packages
import pandas as pd
import numpy as np 
from Bio import Entrez

#searching PubMed for "genomics" and downloading dictionary of data including number of publications, accession IDs, and relevant MeSH term 
Entrez.email = "eileen.cahill@nih.gov"
handle = Entrez.esearch(db="pubmed", retmax=100, term="genom*", idtype="acc")
record = Entrez.read(handle)
handle.close()

print(record)
if record['Count'] == '0':
    print("\nYour search did not return any publications. Try another search.\n")
    print("Error list says: " + str(record['ErrorList']) + "\n")
    print("Warning list says: " + str(record['WarningList']))
else:
    print("Search successful.")

l Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiprojektin[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomique[All Fields]', 'Field': 'All Fields', 'Count': '4304', 'Explode': 'N'}, 'OR', {'Term': "genomique'[All Fields]", 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiqueet[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiqueevry[All Fields]', 'Field': 'All Fields', 'Count': '2', 'Explode': 'N'}, 'OR', {'Term': 'genomiquefonctionnelle[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiques[All Fields]', 'Field': 'All Fields', 'Count': '169', 'Explode': 'N'}, 'OR', {'Term': 'genomiquespublishing[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomircs[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomis[All Fields]', 'Field'

In [3]:
#looking at data and types
print(type(record))
print(type(handle))
#print(record)
print(record['IdList'])


<class 'Bio.Entrez.Parser.DictionaryElement'>
<class '_io.TextIOWrapper'>
['32369867', '32369848', '32369831', '32369821', '32369817', '32369810', '32369809', '32369805', '32369735', '32369734', '32369668', '32369633', '32369600', '32369593', '32369588', '32369585', '32369572', '32369566', '32369565', '32369554', '32369553', '32369552', '32369522', '32369498', '32369496', '32369493', '32369486', '32369481', '32369457', '32369456', '32369452', '32369445', '32369358', '32369273', '32369165', '32369034', '32369020', '32369019', '32369005', '32369004', '32369003', '32369001', '32369000', '32368999', '32368997', '32368983', '32368982', '32368974', '32368927', '32368866', '32368861', '32368792', '32368734', '32368696', '32368685', '32368569', '32368540', '32368513', '32368431', '32368353', '32368352', '32368345', '32368326', '32368312', '32368310', '32368301', '32368296', '32368197', '32368194', '32368185', '32368168', '32368150', '32368149', '32368108', '32368105', '32368102', '32368101', '

In [4]:
#converting type from Bio.Entrez.Parser.DictionaryElement to list
list = []
listrecords = []

for key, value in record.items():
    list = [key, value]
    listrecords.append(list)
print(listrecords)
print(type(listrecords))

 Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiprojektin[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomique[All Fields]', 'Field': 'All Fields', 'Count': '4304', 'Explode': 'N'}, 'OR', {'Term': "genomique'[All Fields]", 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiqueet[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiqueevry[All Fields]', 'Field': 'All Fields', 'Count': '2', 'Explode': 'N'}, 'OR', {'Term': 'genomiquefonctionnelle[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomiques[All Fields]', 'Field': 'All Fields', 'Count': '169', 'Explode': 'N'}, 'OR', {'Term': 'genomiquespublishing[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomircs[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'OR', {'Term': 'genomis[All Fields]', 'Field':

In [5]:
#creating iterable list of IDs of both string and int type
id_list_acquire = listrecords[3]
print(id_list_acquire)
id_list = []
id_list_int = []

for id in id_list_acquire[1]:
    id_list.append(id)
print(id_list)

['IdList', ['32369867', '32369848', '32369831', '32369821', '32369817', '32369810', '32369809', '32369805', '32369735', '32369734', '32369668', '32369633', '32369600', '32369593', '32369588', '32369585', '32369572', '32369566', '32369565', '32369554', '32369553', '32369552', '32369522', '32369498', '32369496', '32369493', '32369486', '32369481', '32369457', '32369456', '32369452', '32369445', '32369358', '32369273', '32369165', '32369034', '32369020', '32369019', '32369005', '32369004', '32369003', '32369001', '32369000', '32368999', '32368997', '32368983', '32368982', '32368974', '32368927', '32368866', '32368861', '32368792', '32368734', '32368696', '32368685', '32368569', '32368540', '32368513', '32368431', '32368353', '32368352', '32368345', '32368326', '32368312', '32368310', '32368301', '32368296', '32368197', '32368194', '32368185', '32368168', '32368150', '32368149', '32368108', '32368105', '32368102', '32368101', '32368095', '32367807', '32367804', '32367802', '32367801', '323

In [6]:
list_titles = []
list_pmids = []

for item in id_list:
    handle2 = Entrez.esummary(db="pubmed", id=item, retmode="xml")
    records2 = Entrez.parse(handle2)

    for record in records2:
        #each record is a Python dictionary or list.
        list_titles.append(record['Title'])

handle2.close()

#print(list_titles)

In [7]:
#showing the dataframe and shape
df = pd.DataFrame(listrecords)
print(df)
df.shape

df = pd.DataFrame(list_titles)
print(df)
df.shape

0                                                  1
0             Count                                             799431
1            RetMax                                                100
2          RetStart                                                  0
3            IdList  [32369867, 32369848, 32369831, 32369821, 32369...
4    TranslationSet                                                 []
5  TranslationStack  [{'Term': 'genom[All Fields]', 'Field': 'All F...
6  QueryTranslation  genom[All Fields] OR genom'ic[All Fields] OR g...
7       WarningList  {'OutputMessage': ['Wildcard search for 'genom...
                                                    0
0   5' splice site GC&gt;GT and GT&gt;GC variants ...
1   Platelet Integrin αIIbβ3 Activation is Associa...
2   Increased urinary exosomal SYT17 levels in chr...
3   Mutational Profiling of Driver Tumor Suppresso...
4   What Will It Take to Build an Expert Group of ...
..                                                ...
9

(100, 1)

In [8]:
text_glob = ""

for x in list_titles:
    text_glob += x

text_glob = text_glob.replace(".", " ")
text_glob = text_glob.replace(",", " ")
text_glob = text_glob.replace(":", " ")
text_glob = text_glob.replace(";", " ")
text_glob = text_glob.replace("(", " ")
text_glob = text_glob.replace(")", " ")
text_glob = text_glob.replace("?", " ")
text_glob = text_glob.replace("&", " ")
text_glob = text_glob.replace("'", " ")
text_glob = text_glob.replace("<", " ")
text_glob = text_glob.replace(">", " ")
text_glob = text_glob.replace("/i", " ")

text_lower = text_glob.lower()
print(text_lower)

5  splice site gc gt gt and gt gt gc variants differ markedly in terms of their functionality and pathogenicity platelet integrin αiibβ3 activation is associated with 25-hydroxyvitamin d concentrations in healthy adults increased urinary exosomal syt17 levels in chronic active antibody-mediated rejection after kidney transplantation via the il-6 amplifier mutational profiling of driver tumor suppressor and oncogenic genes in brazilian malignant pleural mesotheliomas what will it take to build an expert group of nutrigenomic practitioners nonmosaic trisomy 19p13 3p13 2 resulting from a rare unbalanced t y 19  q12 p13 2  translocation in a patient with pachygyria and polymicrogyria introducing the bird chromosome database  an overview of cytogenetic studies in birds cytogenetic and molecular characterization of three mimetic species of the genus alagoasa bechyné 1955  coleoptera  alticinae  from the neotropical region nad sup + /sup  controls circadian reprogramming through per2 nuclear 

In [9]:
#tokenization
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer

#Create word tokens and count frequencies
tokens = word_tokenize(text_lower)
#print(tokens)

#removing stopwords and showing most frequent non-stopword words
stops = set(stopwords.words('english'))
stopwords = [x for x in tokens if x not in stops]
#print(stopwords)

freq_words = FreqDist(stopwords)
#for word in freq_words:
    #print(word, freq_words[word])

most_freq_words = freq_words.most_common(15)
print(most_freq_words)

[('cell', 8), ('sp', 8), ('genetic', 8), ('nov', 8), ('virus', 8), ('cancer', 8), ('gene', 7), ('novel', 6), ('analysis', 6), ('genes', 5), ('characterization', 5), ('using', 5), ('development', 5), ('sequencing', 5), ('expression', 5)]


In [85]:
#Named Entity Recognition
from nltk import pos_tag
from nltk import ne_chunk

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

tags = pos_tag(stopwords)
chunk = ne_chunk(tags)

#print(chunk)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eileencahill/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/eileencahill/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/eileencahill/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [91]:
#Establishing rules for chunking
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *


rule1 = 'NP: {<DT>?<JJ>*<NN.*>}'
chunkparser = RegexpParser(rule1)
result = chunkparser.parse(tags)
print(result)


(S
  5/CD
  (NP splice/NN)
  (NP site/NN)
  (NP gc/NN)
  (NP gt/NN)
  (NP gt/NN)
  (NP gt/NN)
  (NP gt/NN)
  (NP gc/NN)
  (NP variants/NNS)
  differ/VBP
  markedly/RB
  (NP terms/NNS)
  functionality/VBP
  (NP pathogenicity/NN)
  (NP platelet/NN)
  (NP integrin/NN)
  (NP αiibβ3/NNP)
  (NP activation/NN)
  associated/VBN
  (NP 25-hydroxyvitamin/JJ concentrations/NNS)
  (NP healthy/JJ adults/NNS)
  increased/VBD
  (NP urinary/JJ exosomal/JJ syt17/NN)
  (NP levels/NNS)
  (NP chronic/JJ active/JJ antibody-mediated/JJ rejection/NN)
  (NP kidney/NN)
  (NP transplantation/NN)
  via/IN
  (NP il-6/JJ amplifier/JJ mutational/JJ profiling/NN)
  (NP driver/NN)
  (NP tumor/NN)
  (NP suppressor/NN)
  (NP oncogenic/JJ genes/NNS)
  (NP brazilian/JJ malignant/JJ pleural/JJ mesotheliomas/NNS)
  take/VBP
  (NP build/JJ expert/NN)
  (NP group/NN)
  (NP nutrigenomic/JJ practitioners/NNS)
  nonmosaic/VBP
  trisomy/JJ
  19p13/CD
  3p13/CD
  2/CD
  resulting/VBG
  rare/JJ
  unbalanced/JJ
  19/CD
  (NP q12/NN)

In [34]:
reg = NP: "{<DT>|<JJ>*<NN>}” 
a = nltk.RegexpParser(reg)
result = a.parse(tags)
print(result)

SyntaxError: invalid syntax (<ipython-input-34-b0772fdf9997>, line 1)

## 7) Give me a problem statement.
Below, write a problem statement. Keep in mind that your task is to tease out relationships in your data and eventually build a predictive model. Your problem statement can be vague, but you should have a goal in mind. Your problem statement should be between one sentence and one paragraph.

In [14]:
#It will be helpful for NIH administrators to know which aspects of a certain scientific area are most common in publications and which are least common. For example if we know that "human" is the most common term in a PubMed search of "genomics" publications, then we can learn that there is a strong human emphasis in genomic research.

## 8) What is your _y_-variable?
For final project, you will need to perform a statistical model. This means you will have to accurately predict some y-variable for some combination of x-variables. From your problem statement in part 7, what is that y-variable?

In [15]:
#How many times a word is used in publication abstract/titles.